In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import bcrypt
import os

In [3]:
database_file = "users.csv"
current_user = None  

def load_users():
    if os.path.exists(database_file):
        return pd.read_csv(database_file)
    return pd.DataFrame(columns=["USERNAME", "PASSWORD"])

def save_users(database):
    database.to_csv(database_file, index=False)

database = load_users()

def user_pass():
    global database
    name = input("Please enter your name to begin: ").strip().upper()
    print(f"WELCOME TO {name}'s PERSONAL TASK MANAGER")

    username = input("Create a unique username: ").strip()
    if not database.empty and username in database["USERNAME"].values:
        print("Username taken. Pick a different username or try logging in.")
        return

    while True:
        pin = input("Create a three-digit PIN to secure your account: ").strip()
        if pin.isdigit() and len(pin) == 3:
            hashed = bcrypt.hashpw(pin.encode(), bcrypt.gensalt()).decode()
            break
        else:
            print("Invalid input. Please enter a three-digit number.")

    print(f"Please remember your credentials (CASE SENSITIVE):\nUsername: {username}\nPIN: {pin}")
    database = pd.concat([database, pd.DataFrame([[username, hashed]], columns=["USERNAME", "PASSWORD"])], ignore_index=True)
    save_users(database)
    login()

In [5]:
def login():
    global current_user
    print("\nLOGIN")

    for _ in range(3):
        username = input("Enter your username: ").strip()
        pin = input("Enter your PIN: ").strip()

        if not database.empty and username in database["USERNAME"].values:
            stored_hashed_pin = database.loc[database["USERNAME"] == username, "PASSWORD"].values[0]
            if bcrypt.checkpw(pin.encode(), stored_hashed_pin.encode()):
                print("ACCESS GRANTED")
                current_user = username  
                services()
                return
            else:
                print("Incorrect Password. Try again.")
        else:
            print("Incorrect Username. Try again.")

    print("Too many failed attempts. Returning to main menu.")

In [7]:
def get_task_file():
    return f"{current_user}_tasks.csv"

def load_tasks():
    task_file = get_task_file()
    if os.path.exists(task_file):
        return pd.read_csv(task_file)
    return pd.DataFrame(columns=["USERNAME", "DATE", "DESCRIPTION", "TASK_ID", "STATUS"])

def save_tasks(task_log):
    task_log.to_csv(get_task_file(), index=False)

def add_task():
    global current_user
    if not current_user:
        print("Please log in first.")
        return

    task_log = load_tasks()
    task_date = input("Enter the date in yyyy-mm-dd format: ")
    task_date = datetime.strptime(task_date, "%Y-%m-%d").date()
    task_des = input("Task description: ").upper()
    task_id_gen = np.random.randint(1, 9999999)

    new_task = pd.DataFrame([[current_user, task_date, task_des, task_id_gen, "PENDING"]],
                             columns=["USERNAME", "DATE", "DESCRIPTION", "TASK_ID", "STATUS"])
    task_log = pd.concat([task_log, new_task], ignore_index=True)
    save_tasks(task_log)
    print("Task added successfully!")
    
def view_task():
    global current_user
    if not current_user:
        print("Please log in first.")
        return
    task_log = load_tasks()
    user_tasks = task_log[task_log["USERNAME"] == current_user]
    print(user_tasks if not user_tasks.empty else "No tasks added yet.")

def complete_task():
    global current_user
    if not current_user:
        print("Please log in first.")
        return
    task_log = load_tasks()
    try:
        task_id = int(input("Enter Task ID: "))
        if task_id in task_log.loc[task_log["USERNAME"] == current_user, "TASK_ID"].values:
            task_log.loc[task_log["TASK_ID"] == task_id, "STATUS"] = "COMPLETED"
            save_tasks(task_log)
            print("Status updated to COMPLETED.")
        else:
            print("Task ID not found.")
    except ValueError:
        print("Invalid input. Please enter a valid Task ID.")

def delete_task():
    global current_user
    if not current_user:
        print("Please log in first.")
        return
    task_log = load_tasks()
    try:
        task_id = int(input("Enter Task ID: "))
        if task_id in task_log.loc[task_log["USERNAME"] == current_user, "TASK_ID"].values:
            task_log = task_log[task_log["TASK_ID"] != task_id]
            save_tasks(task_log)
            print("Task deleted successfully.")
        else:
            print("Task ID not found.")
    except ValueError:
        print("Invalid input. Please enter a valid Task ID.")

In [9]:
def services():
    while True:
        service_num = input(
            """This manager allows you to:
            1. Add a task
            2. View your tasks
            3. Mark a task as completed
            4. Delete a task
            5. LOG OUT
            6. TERMINATE PROGRAM
            Press a number to avail the corresponding service: """
        ).strip()
        if service_num == "1":
            add_task()
        elif service_num == "2":
            view_task()
        elif service_num == "3":
            complete_task()
        elif service_num == "4":
            delete_task()
        elif service_num == "5":
            print("LOGGING OUT")
            global current_user
            current_user = None
            manager()
        else:
            print("Invalid input. Please try again.")

In [11]:
def manager():
    print("Welcome to your TASK MANAGER. This program helps you manage your day better.")
    while True:
        action = input("Press R to register if you are a new user. Press L to login if you already have an account: ").strip().upper()
        if action == "L":
            login()
        elif action == "R":
            user_pass()
        else:
            print("Invalid input. Please try again.")

In [13]:
#call manager() to run the program